In [36]:
import pandas as pd
import sklearn
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt

#load the data from the csv to the pandas df


In [54]:
def data_transformer(df, file):
    columns_to_drop = ["imdb_id", "original_title", "overview", "poster_path", "status", "tagline", "title"]
    df.drop(columns=columns_to_drop, inplace=True)
    df["belongs_to_collection"] = np.where(df["belongs_to_collection"].isnull(), 0, 1)
    df["homepage"] = np.where(df["homepage"].isnull(), 0, 1)
    df["original_language"] = np.where(df["original_language"]=="en", 1, 0)
    df['release_date'] = pd.to_datetime(df['release_date'], format='%m/%d/%y')
    df['release_date'] = df['release_date'].apply(lambda x: x.replace(year=x.year-100) if x.year > 2019 else x)

    colList = ["genres"]
    for col in colList:
        df[col] = np.where(df[col].isnull(), "[]", df[col])
        df[col] = df[col].apply(ast.literal_eval)
        df[col] = df[col].apply(lambda x: [str(item["id"]) for item in x])
        mlb = MultiLabelBinarizer()
        newData = mlb.fit_transform(df[col])
        newDf = pd.DataFrame(newData, columns=col+"_" + mlb.classes_)
        df = pd.merge(df, newDf, how='inner', left_index=True, right_index=True)
        df[col] = df[col].apply(len)
        df = df.rename(columns={col: col+"_num"})

    colList = ["production_companies", "spoken_languages", "Keywords", "cast", "crew", "production_countries"]

    for col in colList:
        df[col] = np.where(df[col].isnull(), "[]", df[col])
        df[col] = df[col].apply(ast.literal_eval)
        df[col] = df[col].apply(len)
        df = df.rename(columns={col: col+"_num"})
    excel_file_path = 'clean_'+file+'.xlsx'
    df.to_excel(excel_file_path, index=False)
    return df

In [55]:
df = pd.read_csv('train.csv')
newdf = data_transformer(df, "train")

In [56]:
df = pd.read_csv('test.csv')
newdf = data_transformer(df, "test")